In [31]:
import os
import csv
import heapq
import numpy as np

In [32]:
class Node:
    
    def __init__(self, parent = None, color= None, index = -1, degree = -1):
        
        self.color = color
        self.index = index
        self.degree = degree
        self.parent = parent
        self.child_indices = []
        self.depth = 0
        if parent:
            self.depth = parent.depth+1    
            
    def __repr__(self):
        """Use by calling repr(node)"""
        return "Node(c: " + repr(self.color) + ", idx: " + repr(self.index) + ", deg: " + repr(self.degree) + ")"    
    
    def expand(self, index):
        if index in self.child_indices:
            #print('Index already exists in the children list')
            return False
        else:
            self.child_indices.append(index)
            #print('Node %d: add child index %d' % (self.index, index))
            return True
            
    def set_color(self, color):
        self.color = color
        
    def __lt__(self, other):
        return self.degree > other.degree
    
class Graph:# undirected
    
    def __init__(self, edges=None, root=None):
        self.edges = None
        self.root = None
        self.vertices = dict()#[]# index to the list denotes the vertex index
        #self.vertex_info = []# heapq collecting index and degree
        
    def add_vertex(self, index, child):
        #print('In add_vertex, ', self.vertices ) 
        if index in self.vertices: #index < len(self.vertices):
            self.vertices[index].degree += 1
            #self.vertex_info[index].degree +=1
            #print('In add_vertex, Add degree to vertex %d' % index)
            #print('Index %d, Vertex degree %d' % (index, self.vertices[index].degree))
        else:
            parent, color, degree = None, None, 1
            self.vertices[index] = Node(parent, color, index, degree)
            #self.vertex_info.append(NodeInfo(index, degree))
            #print('Index {}, Node {}'.format(index, self.vertices[index]))
            #print('In add_vertex, Add new vertex %d' % index)
        self.vertices[index].expand(child)
        
        #print('Vertices \n', self.vertices)
        #heapq.heapify(self.vertices)
        #print('Vertex Info\n', self.vertex_info)        
            
    def set_edges(self,edges):
        self.edges = edges
        for edge in edges:
            # build a list of vertices
            self.add_vertex(edge[0], edge[1])
            self.add_vertex(edge[1], edge[0])
            
        #print('Vertices \n', self.vertices)
        #heapq.heapify(self.vertices)
        #print('Vertex Info\n', self.vertex_info)
        
    def print_graph(self):
        for index, vertex in self.vertices.items():
            rep = "Index " + str(index) + ", Node(c: " + str(vertex.color) + ", idx: " + str(vertex.index) + ", deg: " + str(vertex.degree)
            if vertex.child_indices:
                rep = rep + ", children: "+','.join([ str(child) for child in vertex.child_indices]) + ' )'
            print(rep)
                
        

In [69]:
# child have different color than parent
def print_graph(vertices):
    
    for index, vertex in vertices.items():
        rep = "Index " + str(index) + ", Node(c: " + str(vertex.color) + ", idx: " + str(vertex.index) + ", deg: " + str(vertex.degree)
        if vertex.child_indices:
            rep = rep + ", children: "+','.join([ str(child) for child in vertex.child_indices]) + ' )'
        print(rep)

def get_color(colors, lower_bound=None, upper_bound=None):
    
    color = None
    if not colors: color = 0
    else: color = max(colors)+1
    
    #if color <= lower_bound or color > upper_bound:
    if upper_bound and color > upper_bound:
        raise ValueError('Color is below the lower bound or over the upper bound')
        
    return color

def get_usable_color(index, vertices, colors):
    
    child_colors = set([vertices[cidx].color for cidx in vertices[index].child_indices if vertices[cidx].color != None])
    color, usable_colors = None, None
    
    return colors.difference(child_colors)

def update_color_frequency(color, color_frequency):
    if color in color_frequency: color_frequency[color] += 1
    else: color_frequency[color] = 0

def child_node_checking(cindex, pindex, colors, vertices,color_frequency):
    """
        cindex is child index and 
        pindex is parent index
        if child degree > parent degree and if 
        there is only 1 existing color usable for the child node, set that color to child node
    """
    
    if vertices[cindex].color != None: 
        #print('Vertex %d already explored' % index)
        return False
    if vertices[cindex].degree < vertices[pindex].degree:
        return False
    

    usable_colors = get_usable_color(cindex, vertices, colors)
    if len(usable_colors) != 1:  
        return False
    
    print('In child_node_checking, vertex {}, color {}'.format(cindex, vertices[cindex].color))
    #print('-------------------------Use old color for child node')
    color = usable_colors.pop() 
        
    vertices[cindex].color = color
    colors.add(color)    
    
    update_color_frequency(color, color_frequency)
#     if color in color_frequency: color_frequency[color] += 1
#     else: color_frequency[color] = 0
    print('In child_node_checking, parent vertex %d with degree %d, child vertex %d with degree %d set to color %d' %
          (pindex, vertices[pindex].degree, cindex, vertices[cindex].degree, color))
    
    return True       
        
def node_coloring(index, colors, vertices, lower_bound, upper_bound, color_frequency):
    
    print('In node_coloring, vertex {}, color {}'.format(index, vertices[index].color))
    if vertices[index].color != None: 
        #print('Vertex %d already explored' % index)
        return False
        
#     child_colors = set([vertices[cidx].color for cidx in vertices[index].child_indices if vertices[cidx].color != None])
#     color, usable_colors = None, None
    
#     usable_colors = colors.difference(child_colors)
    usable_colors = get_usable_color(index, vertices, colors)
    
    #print('\nIn vertex {}, Num children {}'.format(index, len(vertices[index].child_indices)))
    #print('Colors {}, Child colors {}, Usable color {}'.format(colors,child_colors,usable_colors))
        
    if len(usable_colors) > 1:    
        print('-------------------------More than 1 color is available to use')
        return False
#     elif not colors:
#         print('-------------------------Get new color')
#         color = get_color(colors, lower_bound, upper_bound)
    elif not usable_colors:
        return False
    else:
        print('-------------------------Use old color')
        color = usable_colors.pop() 
        
    vertices[index].color = color
    colors.add(color)    
    
    update_color_frequency(color, color_frequency)
#     if color in color_frequency: color_frequency[color] += 1
#     else: color_frequency[color] = 0
    print('In node_coloring, vertex %d with degree %d set to color %d' % (index, vertices[index].degree, color))
    
    return True

def graph_coloring_probe(colors, vertices, color_frequency, explored):
    
    vertex_info = []
    for idx, v in vertices.items():
        heapq.heappush(vertex_info, (-v.degree, v.index))
        #vertex_info.append((-v.degree, v.index))
        
    lower_bound, upper_bound = -vertex_info[0][0], len(vertex_info)
    #print('Lower c bound %d, higher bound %d ' % (lower_bound, upper_bound))
    
    print('In graph_coloring_probe, info:{}'.format(vertex_info))
    while vertex_info:

        vinfo = heapq.heappop(vertex_info)
        #print('\n\n New vertex %d with degree %d' % (vinfo[1], -vinfo[0]))
        print('In graph_coloring_probe, pop:{}, info:{}'.format(vinfo, vertex_info))
        
        index = vinfo[1]
        if index in explored: continue
            
        flag = node_coloring(index, colors, vertices, lower_bound, upper_bound, color_frequency)
        if flag: explored.add(index)
            
        for cidx in vertices[index].child_indices:
            # if there is a child with higher degree and can only have 1 value to assigned
            flag = child_node_checking(cidx, index, colors, vertices,color_frequency)
            if flag: explored.add(cidx)
            #flag = node_coloring(cidx, colors, vertices, lower_bound, upper_bound, color_frequency)
            #if flag: explored.add(cidx)
                
        #print('Explored node indices ', explored)
        #print('Used colors ', colors)
        #print_graph(vertices)    
    
def graph_coloring(vertices):
    
    explored = set()
    colors = set()
    color_frequency = dict()
    #heapq.heapify(vertex_info)
    vertex_info = []
    for idx, v in vertices.items():
        heapq.heappush(vertex_info, (-v.degree, v.index))
        #vertex_info.append((-v.degree, v.index))
        
    selected_colors = [v.color for i, v in vertices.items()]
    while not all(selected_colors) and vertex_info:
        vinfo = heapq.heappop(vertex_info)
        print('In graph_coloring, pop:{}, info:{}'.format(vinfo, vertex_info))
        index = vinfo[1]
        if vertices[index].color != None:
            continue
        else:
            color = get_color(colors)
            vertices[index].color = color
            print('In graph_coloring, index {} with degree {} set color {}'.format(index, vertices[index].degree, vertices[index].color))
            colors.add(color) 
            explored.add(index)
            update_color_frequency(color, color_frequency)
            graph_coloring_probe(colors, vertices, color_frequency, explored)
        selected_colors = [v.color for i, v in vertices.items()]
    #print('Original vertices\n', vertices)
    #print('Sorted vertex info\n', vertex_info)
    
    #selected_colors = [vertices[i].color for i in range(len(vertices))]
    return len(colors), [vertices[i].color for i in range(len(vertices))]#, vertices, color_frequency

In [71]:
def solve_it(input_data):
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')

    first_line = lines[0].split()
    node_count = int(first_line[0])
    edge_count = int(first_line[1])
    #print('Node count %d, edge count %d' % (node_count, edge_count))

    edges = []
    for i in range(1, edge_count + 1):
        line = lines[i]
        parts = line.split()
        edges.append((int(parts[0]), int(parts[1])))
        
    #### play with graph
    graph = Graph()
    graph.set_edges(edges)
    #graph.print_graph()
    print('---------------Start graph coloring ------------------')
    #num_colors, solution, vertices, color_frequency = graph_coloring(graph.vertices)
    num_colors, solution = graph_coloring(graph.vertices)
    #print('Vertices ', vertices)
    #vertex_colors = [(v.index, v.color) for k, v in vertices.items()]
    #print('Vertex color', [(v.index, v.color) for v in vertices])
    
    #######
    # build a trivial solution
    # every node has its own color

    # prepare the solution in the specified output format
    output_data = str(num_colors) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, solution))

    return output_data#, vertex_colors, color_frequency


In [73]:
data_folder = 'C:/D/coursera/discrete_opt/coloring/data'
filename = os.path.join(data_folder,'gc_100_7')
with open(filename, 'r') as input_data_file:
    input_data = input_data_file.read()
    
output=solve_it(input_data)
#print('\n\nVertex colors\n', vertex_colors)
#print('\n\nColors frequence\n', color_frequency)
print('\n\n Output\n', output)

---------------Start graph coloring ------------------
In graph_coloring, pop:(-79, 74), info:[(-77, 24), (-77, 69), (-77, 47), (-75, 64), (-75, 80), (-76, 29), (-76, 56), (-74, 8), (-72, 35), (-73, 9), (-74, 89), (-74, 14), (-75, 54), (-75, 2), (-74, 62), (-71, 17), (-74, 32), (-71, 73), (-72, 76), (-73, 20), (-70, 19), (-72, 88), (-72, 92), (-74, 48), (-69, 23), (-69, 52), (-71, 25), (-68, 27), (-68, 57), (-69, 60), (-73, 0), (-70, 63), (-66, 15), (-70, 34), (-74, 70), (-69, 72), (-70, 71), (-70, 75), (-71, 78), (-70, 40), (-72, 81), (-68, 84), (-69, 86), (-68, 44), (-71, 43), (-70, 45), (-68, 46), (-69, 96), (-69, 5), (-69, 49), (-61, 50), (-63, 12), (-65, 51), (-64, 26), (-69, 53), (-63, 55), (-65, 13), (-67, 28), (-60, 58), (-66, 59), (-67, 6), (-59, 30), (-67, 61), (-65, 3), (-68, 31), (-64, 65), (-60, 66), (-63, 16), (-69, 67), (-65, 33), (-71, 68), (-67, 7), (-68, 4), (-65, 36), (-67, 99), (-63, 18), (-67, 38), (-66, 37), (-68, 77), (-57, 79), (-65, 1), (-69, 39), (-67, 82), (-

In [30]:
data_folder = 'C:/D/coursera/discrete_opt/coloring/data'
filename = os.path.join(data_folder,'gc_1000_7')
with open(filename, 'r') as input_data_file:
    input_data = input_data_file.read()
    
output, vertex_colors, color_frequency = solve_it(input_data)
print('\n\nVertex colors\n', vertex_colors)
print('\n\nColors frequence\n', color_frequency)
print('\n\n Output\n', output)

---------------Start graph coloring ------------------
In node_coloring, vertex 579, color None
-------------------------Get new color
In node_coloring, vertex 579 with degree 743 set to color 0
In node_coloring, vertex 176, color None
In node_coloring, vertex 524, color None
In node_coloring, vertex 930, color None
In node_coloring, vertex 636, color None
In node_coloring, vertex 77, color None
-------------------------Use old color
In node_coloring, vertex 77 with degree 733 set to color 0
In node_coloring, vertex 220, color None
In node_coloring, vertex 680, color None
In node_coloring, vertex 754, color None
In node_coloring, vertex 6, color None
-------------------------Use old color
In node_coloring, vertex 6 with degree 731 set to color 0
In node_coloring, vertex 500, color None
In node_coloring, vertex 977, color None
In node_coloring, vertex 96, color None
In node_coloring, vertex 800, color None
In node_coloring, vertex 60, color None
In node_coloring, vertex 206, color None
